In [1]:
from lib.outliers import Runner
from datetime import date
import pandas as pd

In [2]:
from_date = date(year=2021,month=4,day=1)
to_date = date(year=2021,month=8,day=1)
r = Runner(from_date,to_date,5,["practice","ccg","pcn","stp"],False)

In [3]:
r.build.run()
r.build.fetch_results()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=725825577420-unm2gnkiprugilg743tkbig250f4sfsj.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=B0aBfO2cFTgPTpWIrbEXhCszrTmcNv&prompt=consent&access_type=offline


Enter the authorization code:  4/1AX4XfWjWfGmWhSn3IUgFJA9Y1gOE418Hgdc8PD98NKa2Y2AW1-2axRotGlg


Downloading: 100%|██████████| 1/1 [00:00<00:00,  4.85rows/s]


In [14]:
### Extracting all the stored z scores etc across organisations
### so that summary statistics can be calculated

e_data = pd.concat(
    (d.assign(entity=e) for e, d in r.build.results.items())
)

## Entity counts

Counts of each kind of entity (i.e., organisation).

In [15]:
### Summarising the number of each kind of entity (organisation)

e_counts = ( e_data.reset_index()[["practice","entity"]]
            .drop_duplicates()['entity']
            .value_counts()
            .to_frame()
            .rename( columns={'entity':'n'} ) )

e_counts

,n
practice,6499
pcn,1257
ccg,106
stp,42


## Chemical counts

Counts of the number of unique outlying chemicals (i.e., those identified in the top/bottom
5 z scores) amongst all organisations of the given type.

In [16]:
### Summarising the number of unique chemicals identified in the
### top/bottom five outliers amongst all organisations of the given type

c_counts = ( e_data.reset_index()[["chemical","entity"]]
            .drop_duplicates()['entity']
            .value_counts()
            .to_frame()
            .rename( columns={'entity':'chemicals'} ) )

c_counts

,chemicals
pcn,1294
practice,1274
ccg,706
stp,364


In [17]:
### Combining the entity and chemical counts

all_counts = e_counts.join( c_counts )

In [18]:
### Defining which metrics will be displayed in the summary tables
metrics_to_show = [ "n", "chemicals", "median","max","min","IQR" ]

In [19]:
### Calculating summary statistics for the Z scores for those chemicals
### identified in the TOP 5 in at least one organisation of the entity type.
### There are the chemicals displayed in the 'Higher than most' table.

overused_summary = e_data.query('z_score>0').groupby( "entity" )[["z_score"]].describe().reindex(['stp', 'ccg', 'pcn', 'practice']).stack(level=0)
overused_summary = overused_summary.rename( columns={"50%":"median"}, inplace=False )

In [20]:
### Calculating summary statistics for the Z scores for those chemicals
### identified in the BOTTOM 5 in at least one organisation of the entity type.
### There are the chemicals displayed in the 'Lower than most' table.

underused_summary = e_data.query('z_score<0').groupby( "entity" )[["z_score"]].describe().reindex(['stp', 'ccg', 'pcn', 'practice']).stack(level=0)
underused_summary = underused_summary.rename( columns={"50%":"median"}, inplace=False )

## Summary statistics for outlying Z scores in each organisation type

### Higher than most chemicals

The table below summarises the Z scores for the high outlying (i.e., top 5) chemicals
in each type of organisation. These are chemicals are seen to be used more often
in a particular organisation than its peers.

In [21]:
### Extracting the summary statistics for the z scores
overused_tmp = overused_summary[overused_summary.index.isin(["z_score"], level=1)]

### Calculating IQR, removing the row index and rounding to 2dp
overused_toprint = ( overused_tmp
         .assign( IQR = overused_tmp["75%"]-overused_tmp["25%"] )
         .droplevel(level=1)
         .round(2) )

overused_toprint.join( all_counts )[metrics_to_show]

,n,chemicals,median,max,min,IQR
entity,,,,,,
stp,42,364,6.14,6.33,3.68,0.78
ccg,106,706,7.36,10.20,3.56,3.29
pcn,1257,1294,7.26,543.19,2.58,5.11
practice,6499,1274,7.28,5512.02,1.13,5.96


### Lower than most chemicals

The table below summarises the Z scores for the low outlying (i.e., bottom 5) chemicals
in each type of organisation. These are chemicals are seen to be used less often
in a particular organisation than its peers.

In [22]:
### Extracting the summary statistics for the z scores
underused_tmp = underused_summary[underused_summary.index.isin(["z_score"], level=1)]

### Calculating IQR, removing the row index and rounding to 2dp
underused_toprint = ( underused_tmp
         .assign( IQR = underused_tmp["75%"]-underused_tmp["25%"] )
         .droplevel(level=1)
         .round(2) )

underused_toprint.join( all_counts )[metrics_to_show]

,n,chemicals,median,max,min,IQR
entity,,,,,,
stp,42,364,-2.77,-1.78,-6.33,1.15
ccg,106,706,-2.78,-1.47,-10.20,1.16
pcn,1257,1294,-2.61,-1.49,-141.33,1.03
practice,6499,1274,-2.44,-1.13,-711.87,1.07


### Summary

Below is a summary table that combines the 'Higher than most' and 'Lower than most'
results displayed above.

In [23]:
pd.concat([overused_toprint.join( all_counts )[metrics_to_show],
           underused_toprint[metrics_to_show[2:]]],
          keys=["Higher than most", "Lower than most"],axis=1)

Higher than most                                        \
                        n chemicals median      max   min   IQR   
entity                                                            
stp                    42       364   6.14     6.33  3.68  0.78   
ccg                   106       706   7.36    10.20  3.56  3.29   
pcn                  1257      1294   7.26   543.19  2.58  5.11   
practice             6499      1274   7.28  5512.02  1.13  5.96   

         Lower than most                      
                  median   max     min   IQR  
entity                                        
stp                -2.77 -1.78   -6.33  1.15  
ccg                -2.78 -1.47  -10.20  1.16  
pcn                -2.61 -1.49 -141.33  1.03  
practice           -2.44 -1.13 -711.87  1.07